# Beta分布的矩估计
如果$x_i \sim Beta(\alpha, \beta)$，由于$E(x_i)=\frac{\alpha}{\alpha + \beta}$，而$E(x_i^2)=\frac{\alpha ^2}{(\alpha + \beta)^2}+\frac{\alpha \beta}{(\alpha + \beta)^2 (\alpha+\beta+1)}$，从而我们的矩估计即联立：$$\frac{\hat{\alpha}}{\hat{\alpha} + \hat{\beta}}=\bar{x}$$ $$\frac{\hat{\alpha} ^2}{(\hat{\alpha} + \hat{\beta})^2}+\frac{\hat{\alpha} \hat{\beta}}{(\hat{\alpha} + \hat{\beta})^2 (\hat{\alpha}+\hat{\beta}+1)}=\overline{x^{2}}$$ 即可得到矩估计。在这里，我们将联立方程问题转化为一个最优化问题，即最小化： $$\min_{\hat{\alpha},\hat{\beta}}\left[\frac{\hat{\alpha}}{\hat{\alpha} + \hat{\beta}}-\bar{x} \right]^2+\left[\frac{\hat{\alpha} ^2}{(\hat{\alpha} + \hat{\beta})^2}+\frac{\hat{\alpha} \hat{\beta}}{(\hat{\alpha} + \hat{\beta})^2 (\hat{\alpha}+\hat{\beta}+1)}-\overline{x^{2}}\right]^2$$
我们将会重复抽样、估计M=500次，并根据这500次的结果计算矩估计量的偏差（bias）、标准误(standard error)以及均方误差（mean sqrared error）。

In [1]:
import numpy as np
from numpy import random as nprd
from scipy.optimize import minimize
import scipy as sc

def sampling(a,b,N):
    x=nprd.beta(a,b,N)
    return x

def estimate(x):
    meanx=np.mean(x)
    x2=[xi**2 for xi in x]
    meanx2=np.mean(x2)
    def obj(theta):
        return (theta[0]/(theta[0]+theta[1])-meanx)**2 + ((theta[0]/(theta[0]+theta[1]))**2+(theta[0]*theta[1])/((theta[0]+theta[1])**2*(theta[0]+theta[1]+1))-meanx2)**2
    res=minimize(obj, np.array([1,1]), method='nelder-mead', options={'xtol': 1e-4, 'disp': False})
    return res

M=500 ## simulation times
N=200 ## sample size
a=3
b=1 ## true value
RESULT=np.zeros((M,2), np.float64)
for m in range(M):
    x=sampling(a,b,N)
    res=estimate(x)
    RESULT[m]=res.x

MEAN_RESULT=np.average(RESULT, 0)
BIAS=MEAN_RESULT-np.array([a,b])
STD=np.std(RESULT, 0)
MSE2=np.array([i**2 for i in STD])+np.array([i**2 for i in BIAS])
MSE=np.array([np.sqrt(i) for i in MSE2])
print("Bias = ", BIAS)
print("s.e. = ", STD)
print("RMSE = ", MSE)

Bias =  [ 0.0392831   0.01443088]
s.e. =  [ 0.35045361  0.10847059]
RMSE =  [ 0.3526484   0.10942632]


# Beta分布的极大似然估计
由于Beta分布的对数似然函数为$$\ln \left( \alpha, \beta | x \right)=\sum_{i=1}^N \left[ -\ln (Beta(\alpha,\beta))+(\alpha-1) \ln (x_i) + (\beta-1)\ln (1-x_i) \right]$$
最大化似然函数，或者最小化负的似然函数，即可得到极大似然估计。

In [2]:
import numpy as np
from numpy import random as nprd
from scipy.optimize import minimize
import scipy as sc

def sampling(a,b,N):
    x=nprd.beta(a,b,N)
    return x
    
def estimate(x):
    def log_likelihood(theta):
        likeli=np.array([-1*np.log(sc.special.beta(theta[0],theta[1]))+(theta[0]-1)*np.log(xi)+(theta[1]-1)*np.log(1-xi) for xi in x])
        return -1*np.mean(likeli)
    res=minimize(log_likelihood, np.array([1,1]), method='nelder-mead', options={'xtol': 1e-4, 'disp': False})
    return res

M=500 ## simulation times
N=200 ## sample size
a=3
b=1 ## true value
RESULT=np.zeros((M,2), np.float64)
for m in range(M):
    x=sampling(a,b,N)
    res=estimate(x)
    RESULT[m]=res.x

MEAN_RESULT=np.average(RESULT, 0)
BIAS=MEAN_RESULT-np.array([a,b])
STD=np.std(RESULT, 0)
MSE2=np.array([i**2 for i in STD])+np.array([i**2 for i in BIAS])
MSE=np.array([np.sqrt(i) for i in MSE2])
print("Bias = ", BIAS)
print("s.e. = ", STD)
print("RMSE = ", MSE)

Bias =  [ 0.0677773  0.0104206]
s.e. =  [ 0.31265354  0.0886861 ]
RMSE =  [ 0.31991561  0.08929621]


通过比较，发现尽管Bias有大有效，但是极大似然估计的确比矩估计更有效。

# 区间估计
## 小样本正态总体
如果$x_i\sim N\left(  \mu, \sigma^2 \right)$，那么$\bar{x}\sim N\left(\mu,\frac{\sigma^2}{N}\right)$，或者，如果我们用样本方差替代总体方差，那么有：$$\frac{\bar{x}-\mu}{\sqrt{\frac{s^2}{N}}}\sim t(N-1)$$
因而，对于总体均值$\mu$的95%置信水平的区间估计为$$\left[\bar{x}-t_{1-\alpha}(N-1)\sqrt{\frac{s^2}{N}},\bar{x}+t_{1-\alpha}(N-1)\sqrt{\frac{s^2}{N}}\right]$$
下面我们将重复1000次区间估计，理论上，在95%置信水平下，应该有950次区间估计包含真值，我们将看到这一结论是否成立。

In [3]:
import numpy as np
from numpy import random as nprd
from scipy import special as func

def sampling(mu, sigma2, N):
    x=nprd.normal(mu,np.sqrt(sigma2),N)
    return x

## true value
mu=10
sigma2=30
N=10 # sample size
## iteration times
M=1000
## confidence level
alpha=0.05
## results
included=np.zeros(M)
for i in range(M):
    x=sampling(mu,sigma2,10)
    xmean=np.mean(x)
    xstd=np.std(x)
    lower=xmean-func.stdtrit(N-1,1-alpha/2)*xstd/np.sqrt(N)
    upper=xmean+func.stdtrit(N-1,1-alpha/2)*xstd/np.sqrt(N)
    # 如果包含真值
    if mu>=lower and mu<=upper:
        included[i]=1
print("The prob. of included=",np.mean(included))

The prob. of included= 0.943


# 区间估计
## 大样本总体未知
如果$x_i\sim \left(  \mu, \sigma^2 \right)$，那么当样本量足够大时，$\bar{x}\sim N\left(\mu,\frac{\sigma^2}{N}\right)$，或者，如果我们用样本方差替代总体方差，那么当样本量足够大时，有：$$\frac{\bar{x}-\mu}{\sqrt{\frac{s^2}{N}}}\sim N\left(0,1\right)$$
下面我们将重复1000次区间估计，理论上，在95%置信水平下，应该有950次区间估计包含真值，我们将看到这一结论是否成立。

In [4]:
import numpy as np
from numpy import random as nprd
from scipy import special as func

def sampling(N):
    ## 产生一组样本，以0.5的概率为z+3，0.5的概率为z-3，其中z~N(0,1)，因而期望为0，方差为10
    d=nprd.rand(N)<0.5
    z=nprd.randn(N)
    x=np.array([z[i]+3 if d[i] else z[i]-3 for i in range(N)])
    return x

## true value
N=[10,50,100,500,1000] # sample size
## iteration times
M=1000
## confidence level
alpha=0.05
for n in N:
    ## results
    included=np.zeros(M)
    for i in range(M):
        x=sampling(n)
        xmean=np.mean(x)
        xstd=np.std(x)
        lower=xmean-func.ndtri(1-alpha/2)*xstd/np.sqrt(n)
        upper=xmean+func.ndtri(1-alpha/2)*xstd/np.sqrt(n)
        # 如果包含真值
        #print(mu, lower, upper, mu>=lower, mu<=upper)
        if 0>=lower and 0<=upper:
            included[i]=1
    print("Sample size=%s, the prob. of included=" % n, np.mean(included))

Sample size=10, the prob. of included= 0.909
Sample size=50, the prob. of included= 0.951
Sample size=100, the prob. of included= 0.95
Sample size=500, the prob. of included= 0.942
Sample size=1000, the prob. of included= 0.95
